In [ ]:
from google.colab import drive

# Mount Google Drive to access the dataset
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.applications import VGG16, DenseNet121, ResNet50
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Define paths
data_dir = "/content/drive/MyDrive/dataset2"

# Image data generator for loading images
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Constants
img_height, img_width = 224, 224
batch_size = 32

# Create data generators for training and validation
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Define a Convolutional LSTM model
model_clstm = Sequential([
    ConvLSTM2D(64, (3, 3), input_shape=(None, img_height, img_width, 3), padding='same', return_sequences=True),
    ConvLSTM2D(64, (3, 3), padding='same', return_sequences=False),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])

model_clstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 5
history_clstm = model_clstm.fit(train_generator, epochs=epochs, validation_data=val_generator)

# Evaluate the model
val_generator.reset()
clstm_predictions = model_clstm.predict(val_generator)
clstm_report = classification_report(val_generator.classes, np.argmax(clstm_predictions, axis=1))
print("ConvLSTM Classification Report:\n", clstm_report)


Found 2445 images belonging to 3 classes.
Found 609 images belonging to 3 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
